## **Setting up the Development Environment**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 2

/content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 2


In [3]:
! pip install -U -q transformers
! pip install -U -q sentencepiece

## **Importing Libraries**

In [4]:
import pandas as pd

import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from transformers import RobertaTokenizer, RobertaModel

import pickle

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## **Loading Data**

In [5]:
dataframe = pd.read_csv("Dataset/hasoc_tamil_task2_withlabels.tsv", sep='\t')

In [6]:
dataframe.head()

,id,text,category
0,Tam_1,Indha movie ku award tharlana avanga mansanay ...,OFF
1,Tam_2,kritheeck Kookaburra en unaku enachu? Cbsc ah??,NOT
2,Tam_3,Actually Oru particular bus incident thalaiya ...,OFF
3,Tam_4,Small suggestions: mic ah shirt la pottukunga bro,NOT
4,Tam_5,Karnan padathulaa oru pombaa varumlaa athuu en...,NOT


In [7]:
dataframe.describe()

,id,text,category
count,1001,1001,1000
unique,1001,994,2
top,Tam_594,Indha movie ku award tharlana avanga mansanay ...,NOT
freq,1,3,605


In [8]:
dataframe['category'].unique()

array(['OFF', 'NOT', nan], dtype=object)

In [9]:
dataframe.shape

(1001, 3)

In [10]:
dataframe = dataframe.dropna()
dataframe['category'].unique()

array(['OFF', 'NOT'], dtype=object)

In [11]:
dataframe.shape

(1000, 3)

In [12]:
text = dataframe['text']
text

0       Indha movie ku award tharlana avanga mansanay ...
1         kritheeck Kookaburra en unaku enachu? Cbsc ah??
2       Actually Oru particular bus incident thalaiya ...
3       Small suggestions: mic ah shirt la pottukunga bro
4       Karnan padathulaa oru pombaa varumlaa athuu en...
                              ...                        
996     Chai spoiler vaya mooduya full kathayum sillit...
997     Unakku thinga soru irukko illayo aduthavangala...
998     Dai seripu oli advingi mala vandavan thana mau...
999     Gomala bule shirt ta nee padam eduthu paaruda ...
1000    Mr. Maran neenga mindvoice nu nenachu sound ah...
Name: text, Length: 1000, dtype: object

In [13]:
label = dataframe['category']
label

0       OFF
1       NOT
2       OFF
3       NOT
4       NOT
       ... 
996     OFF
997     OFF
998     OFF
999     OFF
1000    NOT
Name: category, Length: 1000, dtype: object

## **Data Preprocessing**

In [14]:
le = LabelEncoder()
label = le.fit_transform(label)
# label

In [15]:
text = text.str.replace(r"[+/#@&*$%:]",'')
text = text.to_numpy()
# text

In [16]:
english_stopwords = stopwords.words("english")
english_stemmer = SnowballStemmer("english")

def clean_text(text):
    tokens = []
    for token in text.split():
        if token not in english_stopwords:
            token = english_stemmer.stem(token)
            tokens.append(token)
    return " ".join(tokens)

text = [clean_text(i) for i in text]

text = np.array(text)

## **Feature Extraction**

In [17]:
# Load Transformer Model

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
# Sample Output

tokenized_input = tokenizer(
        text[0],
        padding=True,
        truncation=False,
        return_tensors='pt'
    )

sample_output = model(**tokenized_input)

In [19]:
print(tokenized_input)

{'input_ids': tensor([[    0,  2028,  1999, 32924,   118,   449,   257,  2354,  3553, 11278,
          1113,  6402,  9615, 36089,   260,   857,  4812,   102, 11051,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [20]:
print(sample_output)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-2.7472e-02,  4.5338e-02, -3.4662e-03,  ..., -8.2291e-02,
          -3.6693e-02,  5.7079e-03],
         [-1.6811e-01, -5.3337e-02,  2.0592e-01,  ..., -6.7219e-01,
           6.7121e-02,  1.7499e-01],
         [ 1.3913e-01,  2.7414e-02, -5.5128e-04,  ..., -3.1488e-01,
           8.4039e-03,  6.6047e-02],
         ...,
         [ 1.2296e-02, -1.0760e-01, -4.3916e-03,  ..., -2.4344e-01,
           1.9754e-02,  6.7114e-02],
         [ 1.0331e-01,  4.8544e-02, -3.4278e-03,  ..., -8.3072e-02,
           2.8795e-02,  6.4635e-02],
         [-1.9503e-02,  3.6827e-02, -1.8291e-02,  ..., -1.1601e-01,
          -3.1386e-02, -9.1136e-03]]], grad_fn=<NativeLayerNormBackward>), pooler_output=tensor([[ 0.0049, -0.2244, -0.2164, -0.1002,  0.1238,  0.1726,  0.2602, -0.0930,
         -0.0512, -0.1612,  0.2540,  0.0095, -0.1064,  0.0869, -0.1413,  0.4850,
          0.2090, -0.4558,  0.0292, -0.0267, -0.2513,  0.0591,  0.4701,  0.3199

In [21]:
print(sample_output.pooler_output.cpu().detach().numpy().shape)

(1, 768)


In [22]:
# Tokenizing Input Data

input = []

for i in text:
    tokenized_input = tokenizer(
        i,
        padding=True,
        truncation=True,
        return_tensors='pt'
    )
    input.append(tokenized_input)

In [23]:
# Embedding the Input Data

output = []

j = 1

for i in range(len(input)):
    
    model_output = model(**input[i])
    model_output = model_output.pooler_output.cpu().detach().numpy()
    output.append(model_output)
    
    # Saving Embedded Input Data to Disk

    if ((i + 1) % 200) == 0:
        
        file_name = "./XLM-Roberta-Embedded-TrainData/output" + str(j) + ".pickle"
        
        with open(file_name, "wb") as fp:   
            # Pickling
            pickle.dump(output, fp)
        
        print(file_name + " done")
        output = []
        j += 1
        
file_name = "./XLM-Roberta-Embedded-TrainData/output" + str(j) + ".pickle"

with open(file_name, "wb") as fp:   
    # Pickling
    pickle.dump(output, fp)

print(file_name + " done")
output = []

./XLM-Roberta-Embedded-TrainData/output1.pickle done
./XLM-Roberta-Embedded-TrainData/output2.pickle done
./XLM-Roberta-Embedded-TrainData/output3.pickle done
./XLM-Roberta-Embedded-TrainData/output4.pickle done
./XLM-Roberta-Embedded-TrainData/output5.pickle done
./XLM-Roberta-Embedded-TrainData/output6.pickle done


In [24]:
# Loading Embedded Input Data from Disk

output = []

for i in range(6):
    file_name = "./XLM-Roberta-Embedded-TrainData/output" + str(i + 1) + ".pickle"
    with open(file_name, "rb") as fp:   #Pickling
        file_output = pickle.load(fp)
        for x in file_output:
            output.append(x)
    print(file_name + " done")

X = output
output = []

./XLM-Roberta-Embedded-TrainData/output1.pickle done
./XLM-Roberta-Embedded-TrainData/output2.pickle done
./XLM-Roberta-Embedded-TrainData/output3.pickle done
./XLM-Roberta-Embedded-TrainData/output4.pickle done
./XLM-Roberta-Embedded-TrainData/output5.pickle done
./XLM-Roberta-Embedded-TrainData/output6.pickle done


In [25]:
X = np.array(X)
X.shape

(1000, 1, 768)

In [26]:
X = X.reshape(1000, 768)
X.shape

(1000, 768)

In [27]:
y = label
y.shape

(1000,)

## **Train Test Split**

In [28]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [29]:
X_train.shape

(800, 768)

In [30]:
X_val.shape

(200, 768)

## **Logistic Regression**

In [31]:
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_val)
print(classification_report(y_val, lr_pred))

              precision    recall  f1-score   support

           0       0.65      0.96      0.77       125
           1       0.67      0.13      0.22        75

    accuracy                           0.65       200
   macro avg       0.66      0.55      0.50       200
weighted avg       0.66      0.65      0.57       200



## **Support Vector Machine**

In [32]:
svm = SVC()
svm.fit(X_train, y_train)
svm_pred = svm.predict(X_val)
print(classification_report(y_val, svm_pred))

              precision    recall  f1-score   support

           0       0.62      1.00      0.77       125
           1       0.00      0.00      0.00        75

    accuracy                           0.62       200
   macro avg       0.31      0.50      0.38       200
weighted avg       0.39      0.62      0.48       200



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## **Naive Bayes**

In [33]:
nb = GaussianNB()
nb.fit(X_train, y_train)
nb_pred = nb.predict(X_val)
print(classification_report(y_val, nb_pred))

              precision    recall  f1-score   support

           0       0.68      0.54      0.60       125
           1       0.43      0.57      0.49        75

    accuracy                           0.56       200
   macro avg       0.56      0.56      0.55       200
weighted avg       0.59      0.56      0.56       200



## **Stochastic Gradient Descent**

In [34]:
sgd = SGDClassifier()
sgd.fit(X_train, y_train)
sgd_pred = sgd.predict(X_val)
print(classification_report(y_val, sgd_pred))

              precision    recall  f1-score   support

           0       0.63      1.00      0.77       125
           1       1.00      0.01      0.03        75

    accuracy                           0.63       200
   macro avg       0.81      0.51      0.40       200
weighted avg       0.77      0.63      0.49       200



## **K Nearest Neighbours**

In [35]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_val)
print(classification_report(y_val, knn_pred))

              precision    recall  f1-score   support

           0       0.66      0.66      0.66       125
           1       0.43      0.44      0.44        75

    accuracy                           0.57       200
   macro avg       0.55      0.55      0.55       200
weighted avg       0.58      0.57      0.58       200



## **Decision Tree**

In [36]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_val)
print(classification_report(y_val, dt_pred))

              precision    recall  f1-score   support

           0       0.63      0.58      0.60       125
           1       0.38      0.43      0.40        75

    accuracy                           0.52       200
   macro avg       0.50      0.50      0.50       200
weighted avg       0.53      0.52      0.53       200



## **Random Forest**

In [37]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_val)
print(classification_report(y_val, rf_pred))

              precision    recall  f1-score   support

           0       0.66      0.81      0.73       125
           1       0.49      0.31      0.38        75

    accuracy                           0.62       200
   macro avg       0.57      0.56      0.55       200
weighted avg       0.60      0.62      0.60       200



## **Majority Voting**

In [38]:
mv_pred = []

for i in range(len(lr_pred)):
    one = 0
    zero = 0
    # Only having RF, LR & SGD in predictions 
    # due poor performance of other models
    predictions = [lr_pred[i], sgd_pred[i], rf_pred[i]]
    for pred in predictions:
        if pred == 1: one += 1
        if pred == 0: zero +=1
    if one > zero: mv_pred.append(1)
    else: mv_pred.append(0)

mv_pred = np.array(mv_pred)

print(classification_report(y_val, mv_pred))

              precision    recall  f1-score   support

           0       0.65      0.97      0.78       125
           1       0.71      0.13      0.22        75

    accuracy                           0.66       200
   macro avg       0.68      0.55      0.50       200
weighted avg       0.67      0.66      0.57       200

